In [1]:
'''
TODO: In this cell, describe your choices for each of the following

PEFT technique:
Model:
Evaluation approach:
Fine-tuning dataset:
'''

'\nTODO: In this cell, describe your choices for each of the following\n\nPEFT technique:\nModel:\nEvaluation approach:\nFine-tuning dataset:\n'

In [2]:
'''
Loading and Evaluating a Foundation Model
TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.
'''

'\nLoading and Evaluating a Foundation Model\nTODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.\n'

In [5]:
pip install datasets scikit-learn peft

You should consider upgrading via the 'c:\users\aayus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Using cached install-1.3.5-py3-none-any.whl (3.2 kB)
  Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
  Using cached scikit_learn-1.5.0-cp39-cp39-win_amd64.whl (11.0 MB)
  Using cached peft-0.11.1-py3-none-any.whl (251 kB)
  Using cached multiprocess-0.70.16-py39-none-any.whl (133 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached pyarrow-16.1.0-cp39-cp39-win_amd64.whl (25.9 MB)
  Using cached aiohttp-3.9.5-cp39-cp39-win_amd64.whl (371 kB)
  Using cached xxhash-3.4.1-cp39-cp39-win_amd64.whl (29 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Using cached psutil-5.9.8-cp37-abi3-win_amd64.whl (255 kB)
  Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)
  Using cached safetensors-0.4.3-cp39-none-wi

You should consider upgrading via the 'c:\users\aayus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [6]:
from datasets import load_dataset
#Load the sms_spam dataset
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

c:\users\aayus\appdata\local\programs\python\python39\lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for sms_spam contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sms_spam
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [7]:
dataset["train"][0]

{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n',
 'label': 1}

In [8]:
#preprocess datasets
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\users\aayus\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aayus\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\users\aayus\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [10]:
#load and setup the model
#base model is considered as the foundation model with no parameters adjusted a.k.a freeze parameters

from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in base_model.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(base_model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [21]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return{"accuracy":(predictions==labels).mean()}

In [22]:
#base trainer is just defining the trainer of the foundation model 
base_trainer = Trainer(
    model=base_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate = 2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        # Evaluate and save the model after each epoch
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
        label_names=["labels"]
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [23]:
#this is just to evaluate the accuracy of foundation model with freeze parameters
base_trainer.evaluate()

{'eval_loss': 0.6529563069343567,
 'eval_accuracy': 0.8708520179372198,
 'eval_runtime': 139.1089,
 'eval_samples_per_second': 8.015,
 'eval_steps_per_second': 2.006}

In [24]:
'''
Performing Parameter-Efficient Fine-Tuning¶
TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.
'''

'\nPerforming Parameter-Efficient Fine-Tuning¶\nTODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.\n'

In [25]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(task_type='SEQ_CLS',
                    r=4,
                    target_modules=['classifier'])

In [26]:
# this is the model using Lora Config i.e. using PEFT Tuning
lora_model = get_peft_model(base_model, config)

In [27]:
lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate = 2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        # Evaluate and save the model after each epoch
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
        label_names=["labels"]
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

lora_trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.073400,0.060563,0.988341
2,0.042700,0.058627,0.988341


c:\users\aayus\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\users\aayus\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=2230, training_loss=0.07232361609625709, metrics={'train_runtime': 1691.6046, 'train_samples_per_second': 5.272, 'train_steps_per_second': 1.318, 'total_flos': 102642598010760.0, 'train_loss': 0.07232361609625709, 'epoch': 2.0})

In [28]:
lora_trainer.evaluate()

{'eval_loss': 0.05862682685256004,
 'eval_accuracy': 0.9883408071748879,
 'eval_runtime': 165.2777,
 'eval_samples_per_second': 6.746,
 'eval_steps_per_second': 1.688,
 'epoch': 2.0}

In [32]:
# save the PEFT tuned model i.e. lora model
lora_model.save_pretrained("my_lora_test")

c:\users\aayus\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
# Load the saved PEFT Model
from peft import AutoPeftModelForSequenceClassification, PeftModel
lora_model1 = AutoPeftModelForSequenceClassification.from_pretrained("my_lora_test")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Target modules {'classifier'} not found in the base model. Please check the target modules and try again.